<a href="https://colab.research.google.com/github/suman111999/Breast-Cancer-Classification/blob/main/ANNreg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import sklearn.ensemble
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.model_selection import cross_val_score  
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import itertools
from sklearn.metrics import precision_score, recall_score, confusion_matrix, roc_curve, precision_recall_curve, accuracy_score
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
df=pd.read_excel('BreastCancer_Prognostic_v1.xlsx')
df.head()

,ID,Outcome,Time,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,...,Worst_perimeter,Worst_area,Worst_smoothness,Worst_compactness,Worst_concavity,Worst_concave_points,Worst_symmetry,Worst_fractal_dimension,Tumor_Size,Lymph_Node_Status
0,119513,N,31,18.02,27.60,117.50,1013.0,0.09489,0.1036,0.1086,...,139.70,1436.0,0.1195,0.1926,0.3140,0.1170,0.2677,0.08113,5.0,5
1,8423,N,61,17.99,10.38,122.80,1001.0,0.11840,0.2776,0.3001,...,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,3.0,2
2,842517,N,116,21.37,17.44,137.50,1373.0,0.08836,0.1189,0.1255,...,159.10,1949.0,0.1188,0.3449,0.3414,0.2032,0.4334,0.09067,2.5,0
3,843483,N,123,11.42,20.38,77.58,386.1,0.14250,0.2839,0.2414,...,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,2.0,0
4,843584,R,27,20.29,14.34,135.10,1297.0,0.10030,0.1328,0.1980,...,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,3.5,0


In [ ]:
df1=df[df.Outcome != 'N']

In [ ]:
df1.shape

(47, 35)

In [ ]:
df1.replace(to_replace='?',value=np.nan,inplace=True)

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47 entries, 4 to 196
Data columns (total 35 columns):
ID                           47 non-null int64
Outcome                      47 non-null object
Time                         47 non-null int64
radius_mean                  47 non-null float64
texture_mean                 47 non-null float64
perimeter_mean               47 non-null float64
area_mean                    47 non-null float64
smoothness_mean              47 non-null float64
compactness_mean             47 non-null float64
concavity_mean               47 non-null float64
concave_points_mean          47 non-null float64
symmetry_mean                47 non-null float64
fractal_dimension_mean       47 non-null float64
radius_std_dev               47 non-null float64
texture_std_dev              47 non-null float64
perimeter_std_dev            47 non-null float64
area_std_dev                 47 non-null float64
smoothness_std_dev           47 non-null float64
compactness_std_dev

In [ ]:
df1.isna().values.any()

True

In [ ]:
df1=df1.fillna(df1.mean())

In [ ]:
df1.head()

,ID,Outcome,Time,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,...,Worst_perimeter,Worst_area,Worst_smoothness,Worst_compactness,Worst_concavity,Worst_concave_points,Worst_symmetry,Worst_fractal_dimension,Tumor_Size,Lymph_Node_Status
4,843584,R,27,20.29,14.34,135.10,1297.0,0.1003,0.1328,0.19800,...,152.2,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,3.5,0.0
5,843786,R,77,12.75,15.29,84.60,502.7,0.1189,0.1569,0.16640,...,107.3,733.2,0.1706,0.4196,0.5999,0.1709,0.3485,0.11790,2.5,0.0
7,844582,R,77,13.71,20.83,90.20,577.9,0.1189,0.1645,0.09366,...,110.6,897.0,0.1654,0.3682,0.2678,0.1556,0.3196,0.11510,4.0,10.0
13,847990,R,36,14.54,27.54,96.73,658.8,0.1139,0.1595,0.16390,...,124.1,943.2,0.1678,0.6577,0.7026,0.1712,0.4218,0.13410,6.0,6.0
15,848620,R,10,16.13,20.68,108.10,798.8,0.1170,0.2022,0.17220,...,136.8,1315.0,0.1789,0.4233,0.4784,0.2073,0.3706,0.11420,3.0,1.0


In [ ]:
X = df1.iloc[:, 3:].values
y = df1.iloc[:, 2].values

In [ ]:
#X=pd.DataFrame(X)
#X.head()

In [ ]:
#X.shape

In [ ]:
#X.info()

In [ ]:
#y=pd.DataFrame(y)
#y.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 0)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

Using TensorFlow backend.


In [ ]:
classifier = Sequential()

In [ ]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim=16, init='uniform', activation='relu', input_dim=32))
# Adding dropout to prevent overfitting
classifier.add(Dropout(p=0.1))

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
classifier.add(Dense(output_dim=16, init='uniform', activation='relu'))
# Adding dropout to prevent overfitting
classifier.add(Dropout(p=0.1))

In [ ]:
classifier.add(Dense(output_dim=1, init='uniform', activation='linear'))

In [ ]:
classifier.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_absolute_error'])

In [ ]:
classifier.fit(X_train, y_train, batch_size=100, nb_epoch=150,validation_data=(X_test,y_test))

NameError: name 'classifier' is not defined

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error 

In [ ]:
XGBModel = XGBRegressor()
XGBModel.fit(X_train,y_train , verbose=False)

# Get the mean absolute error on the validation data :
XGBpredictions = XGBModel.predict(X_test)
MAE = mean_absolute_error(y_test , XGBpredictions)
print('XGBoost validation MAE = ',MAE)

XGBoost validation MAE =  26.92263231277466
